In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings

In [6]:
df = pd.read_csv('data\stud.csv')

In [7]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [8]:
X = df.drop(columns='math_score', axis=1)
y = df['math_score']

In [9]:
X, y

(     gender race_ethnicity parental_level_of_education         lunch  \
 0    female        group B           bachelor's degree      standard   
 1    female        group C                some college      standard   
 2    female        group B             master's degree      standard   
 3      male        group A          associate's degree  free/reduced   
 4      male        group C                some college      standard   
 ..      ...            ...                         ...           ...   
 995  female        group E             master's degree      standard   
 996    male        group C                 high school  free/reduced   
 997  female        group C                 high school  free/reduced   
 998  female        group D                some college      standard   
 999  female        group D                some college  free/reduced   
 
     test_preparation_course  reading_score  writing_score  
 0                      none             72             74  
